In [6]:
%load_ext sql
import pandas as pd
import sqlite3 as sql
%sql sqlite:///CourseDataWarehouse.db
conn_dw = sql.connect('CourseDataWarehouse.db')

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
%%sql
SELECT Section, InstructorID, Term
FROM FACT_TABLE
    JOIN INSTRUCTOR_DIMENSION USING(InstructorID)
    JOIN COURSE_DIMENSION USING(CourseOfferingID)
    JOIN TIME_DIMENSION USING(CourseMeetingID)
WHERE Term = 'Spring2018'
GROUP BY Title
ORDER BY SectionsPerCourse DESC
LIMIT 100;

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) no such column: SectionsPerCourse
[SQL: SELECT Section, InstructorID, Term
FROM FACT_TABLE
    JOIN INSTRUCTOR_DIMENSION USING(InstructorID)
    JOIN COURSE_DIMENSION USING(CourseOfferingID)
    JOIN TIME_DIMENSION USING(CourseMeetingID)
WHERE Term = 'Spring2018'
GROUP BY Title
ORDER BY SectionsPerCourse DESC
LIMIT 100;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [3]:
%%sql
SELECT DISTINCT Name,Title,Section
FROM FACT_TABLE
    JOIN INSTRUCTOR_DIMENSION USING(InstructorID)
    JOIN COURSE_DIMENSION USING(CourseOfferingID)
    JOIN TIME_DIMENSION USING(CourseMeetingID)
LIMIT 10


 * sqlite:///CourseDataWarehouse.db
Done.


Name,Title,Section
Rebecca I. Bloch,Human Anatomy & Physiology Lab,B
Gregory Patrick Burke,Human Anatomy & Physiology Lab,B
Michael P. Coyne,Human Anatomy & Physiology Lab,B
Jo Ann Drusbosky,Human Anatomy & Physiology Lab,B
Kara E. Hunter,Human Anatomy & Physiology Lab,B
Arleen N. Kardos,Human Anatomy & Physiology Lab,B
Bruce Bradford,General Biology I Lab,I
Paul Caster,General Biology I Lab,I
Milo W. Peck,General Biology I PLG,A
Scott M Brenner,General Biology I PLG,A


In [159]:
%%sql
SELECT i.Name, Count(DISTINCT CatalogID) as CatalogsAssigned, 
       COUNT(DISTINCT InstructorID) as InstructorCount, ROUND(AVG(Calc.CatalogCount),2) as AvgCatalogPerInstructor,
CASE 
    WHEN Count(DISTINCT CatalogID) > AVG(Calc.CatalogCount) THEN 'Overworked'
    ELSE ''
    END AS 'StaffCheck'
FROM FACT_TABLE
    JOIN TIME_DIMENSION USING(CourseMeetingID)
    JOIN CATALOG_DIMENSION USING(CourseID)
    JOIN INSTRUCTOR_DIMENSION as i USING(InstructorID)
    LEFT JOIN COURSE_DIMENSION USING(CourseOfferingID)
    INNER JOIN (SELECT CourseID, i.Name, COUNT(DISTINCT CatalogID) as CatalogCount 
                FROM FACT_TABLE
                  JOIN CATALOG_DIMENSION USING(CourseID)
                  JOIN INSTRUCTOR_DIMENSION as i USING(InstructorID)
                GROUP BY Name) as Calc
WHERE Term = 'Spring2015'
GROUP BY i.Name
ORDER BY StaffCheck DESC
LIMIT 50;

 * sqlite:///CourseDataWarehouse.db
Done.


Name,CatalogsAssigned,InstructorCount,AvgCatalogPerInstructor,StaffCheck
Kathleen A. Wheeler,6,1,5.15,Overworked
Susan N. Bartos,6,1,5.15,Overworked
Aaron R. Van Dyke,4,1,5.15,
Abdel Illah A. Douda,3,1,5.15,
Ahmed M. Ebrahim,4,1,5.15,
Alan C. Murchie,2,1,5.15,
Alina Zuniga,2,1,5.15,
Alison E. Kris,2,1,5.15,
Alyson Margaret Martin,4,1,5.15,
Amalia I. Rusu,4,1,5.15,


In [91]:
%%sql
SELECT DISTINCT CourseTitle, Section, Term, Name, CatalogID
FROM FACT_TABLE
    JOIN TIME_DIMENSION USING(CourseMeetingID)
    JOIN CATALOG_DIMENSION USING(CourseID)
    JOIN INSTRUCTOR_DIMENSION USING(InstructorID)
    LEFT JOIN COURSE_DIMENSION USING(CourseOfferingID)
WHERE Term = 'Winter2015'


 * sqlite:///CourseDataWarehouse.db
Done.


CourseTitle,Section,Term,Name,CatalogID
The American Century: The United States and the World Since 1900,None,Winter2015,Dawn W Massey,HI 0251
Diversity in the Workplace,None,Winter2015,Sharlene A. McEvoy,MG 0520
Social Entrepreneurship,None,Winter2015,Margaret A. Wills,MG 0531
Collaborative Consultation,None,Winter2015,Jay Krishnamoorthy,PY 0535
Psychoeducational Assessment I: Behavioral Approaches,None,Winter2015,Lisa A. Mainiero,PY 0537
Psychoeducational Assessment III: Clinical Approaches,None,Winter2015,Brian Q. Torff,PY 0540
Independent Study,None,Winter2015,Lisa A. Mainiero,NS 0399
Advanced Nursing Roles for Systems Leadership and Improvement,None,Winter2015,Rose P. Rodrigues,NS 0521
Quality Outcomes Management I,None,Winter2015,Suzanne Chamlin-Richer,NS 0523


In [7]:
%%sql
SELECT sum(COURSE_DIMENSION.Actual) as Num_Students, 

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) near "VIEW": syntax error
[SQL: VIEW TABLE FACT_TABLE;]
(Background on this error at: http://sqlalche.me/e/e3q8)
